In [13]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re

columns = [
    "Name", "Calories", "Category",
    "Tryptophan", "Threonine", "Isoleucine", "Leucine", "Lysine", "Methionine", "Phenylalanine", "Valine", "Histidine",
    "Calcium", "Iron", "Magnesium", "Phosphorus", "Potassium", "Sodium", "Zinc", "Copper", "Manganese", "Selenium", "Fluoride",
    "Vitamin A", "Beta-Carotene", "Alpha-Carotene", "Vitamin D", "Vitamin D2", "Vitamin D3",
    "Vitamin E", "Vitamin K", "Vitamin C", "Vitamin B1", "Vitamin B2", "Vitamin B3",
    "Vitamin B4", "Vitamin B5", "Vitamin B6", "Vitamin B9", "Vitamin B12",
    "vegan", "vegetarian"
]

def convert_to_mg(value_str):
    match = re.match(r"([\d\.]+)(µg|mg|g)", value_str)
    if not match:
        # If no unit found, try to parse as float directly
        try:
            val = float(value_str)
            return val
        except:
            return 0.0

    val = float(match.group(1))
    unit = match.group(2)

    if unit == "µg":
        return val / 1000.0
    elif unit == "g":
        return val * 1000.0
    else:
        return val

def scrape_food_page(food_id):
    base_url = f"https://fitaudit.com/food/{food_id}"
    response = requests.get(base_url)
    if response.status_code != 200:
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the raw name
    name_el = soup.find('h1')
    if not name_el:
        return None
    raw_name = name_el.get_text(strip=True)

    # If the name contains a dash, split and take only the part before it
    if "—" in raw_name:
        scraped_name = raw_name.split("—")[0].strip()
    elif "-" in raw_name:
        scraped_name = raw_name.split("-")[0].strip()
    else:
        scraped_name = raw_name

    data = {col: None for col in columns}

    # Extract calories
    calories_span = soup.find('span', class_='him_bx__legend_text')
    if calories_span:
        calories_text = calories_span.get_text(" ", strip=True)
        m = re.search(r"There are\s+(\d+)\s+calories", calories_text)
        if m:
            data["Calories"] = int(m.group(1))

    # Parse vitamins
    vitamins_tbl = soup.find('table', id='vitamins_tbl')
    if vitamins_tbl:
        for row in vitamins_tbl.find_all('tr'):
            name_td = row.find('td', class_='tbl-name')
            value_td = row.find('td', class_='tbl-value')
            if name_td and value_td:
                vitamin_name_el = name_td.find('a')
                if vitamin_name_el:
                    vitamin_name = vitamin_name_el.get_text(strip=True)
                    value_text = value_td.get_text(strip=True)
                    if vitamin_name in columns:
                        data[vitamin_name] = convert_to_mg(value_text)

    # Parse minerals
    minerals_tbl = soup.find('table', id='minerals_tbl')
    if minerals_tbl:
        for row in minerals_tbl.find_all('tr'):
            name_td = row.find('td', class_='tbl-name')
            value_td = row.find('td', class_='tbl-value')
            if name_td and value_td:
                mineral_name_el = name_td.find('a')
                if mineral_name_el:
                    mineral_name = mineral_name_el.get_text(strip=True)
                    value_text = value_td.get_text(strip=True)
                    if mineral_name in columns:
                        data[mineral_name] = convert_to_mg(value_text)

    # Parse amino acids
    amino_url = f"{base_url}/amino"
    amino_response = requests.get(amino_url)
    if amino_response.status_code == 200:
        amino_soup = BeautifulSoup(amino_response.text, 'html.parser')
        amino_tbl = None
        for table in amino_soup.find_all('table', class_='tbl pr__tbrick'):
            header = table.find('th', colspan='2')
            if header and "Amino acids" in header.get_text():
                amino_tbl = table
                break

        if amino_tbl:
            for row in amino_tbl.find_all('tr'):
                name_td = row.find('td', class_='tbl-name')
                value_td = row.find('td', class_='tbl-value')
                if name_td and value_td:
                    amino_name_el = name_td.find('a')
                    if amino_name_el:
                        amino_name = amino_name_el.get_text(strip=True)
                        value_text = value_td.get_text(strip=True)
                        if amino_name in columns:
                            data[amino_name] = convert_to_mg(value_text)

    return data

In [12]:
food_options = {
    "Apples": {
        "food_id": 114558,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Apricots": {
        "food_id": 114512,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Almonds": {
        "food_id": 123632,
        "Category": "Nut",
        "vegan": True,
        "vegetarian": True
    },
    "Beef": {
        "food_id": 137541,
        "Category": "Meat",
        "vegan": False,
        "vegetarian": False
    },
    "Lamb": {
        "food_id": 148503,
        "Category": "Meat",
        "vegan": False,
        "vegetarian": False
    },
    "Pork": {
        "food_id": 127136,
        "Category": "Meat",
        "vegan": False,
        "vegetarian": False
    },
    "Veal": {
        "food_id": 148722,
        "Category": "Meat",
        "vegan": False,
        "vegetarian": False
    },
    "Asparagus": {
        "food_id": 120582,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Avocado": {
        "food_id": 114660,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Bananas": {
        "food_id": 114679,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Basil": {
        "food_id": 193681,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Bass": {
        "food_id": 132010,
        "Category": "Meat",
        "vegan": False,
        "vegetarian": False
    },
    "BayLeaf": {
        "food_id": 193561,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Beer": {
        "food_id": 139558,
        "Category": "Alcohol",
        "vegan": True,
        "vegetarian": True
    },
    "Beets": {
        "food_id": 120799,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "BlackBeans": {
        "food_id": 135594,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "Blackberry": {
        "food_id": 114675,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "BlueCheese": {
        "food_id": 190561,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Blueberries": {
        "food_id": 114609,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Bri": {
        "food_id": 190567,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Broccoli": {
        "food_id": 120729,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "BrownMushrooms": {
        "food_id": 120247,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "BrownSugar": {
        "food_id": 145551,
        "Category": "Carbohydrate",
        "vegan": True,
        "vegetarian": True
    },
    "BrusselsSprouts": {
        "food_id": 120743,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Buckwheat": {
        "food_id": 157589,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "ButterOil": {
        "food_id": 190558,
        "Category": "Oil",
        "vegan": False,
        "vegetarian": True
    },
    "ButterHeadLettuce": {
        "food_id": 120209,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "ButterMilk": {
        "food_id": 190149,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Butterscotch": {
        "food_id": 144769,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Cabbage": {
        "food_id": 120876,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "NoCaffeineCola": {
        "food_id": 139990,
        "Category": "Drink",
        "vegan": True,
        "vegetarian": True
    },
    "ChiaSeeds": {
        "food_id": 123567,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "Chicken": {
        "food_id": 102585,
        "Category": "Meat",
        "vegan": False,
        "vegetarian": False
    },
    "Chickpeas": {
        "food_id": 135617,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "Chives": {
        "food_id": 120917,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Cinnamon": {
        "food_id": 193589,
        "Category": "Spice",
        "vegan": True,
        "vegetarian": True
    },
    "Clementines": {
        "food_id": 115748,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "CocoaButter": {
        "food_id": 100052,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "CoconutMilk": {
        "food_id": 123800,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "CoconutMeat": {
        "food_id": 123861,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "CoconutOil": {
        "food_id": 109690,
        "Category": "Oil",
        "vegan": True,
        "vegetarian": True
    },
    "CoconutWater": {
        "food_id": 123806,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Cod": {
        "food_id": 132356,
        "Category": "Meat",
        "vegan": False,
        "vegetarian": False
    },
    "CannedCorn": {
        "food_id": 120065,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "CannedGreanPeas": {
        "food_id": 122988,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "CanolaOil": {
        "food_id": 100295,
        "Category": "Oil",
        "vegan": True,
        "vegetarian": True
    },
    "CantaloupeMelon": {
        "food_id": 114092,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Capers": {
        "food_id": 193611,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Caramel": {
        "food_id": 144771,
        "Category": "Sweets",
        "vegan": False,
        "vegetarian": True
    },
    "Carrrots": {
        "food_id": 120821,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Cashews": {
        "food_id": 123710,
        "Category": "Nuts",
        "vegan": True,
        "vegetarian": True
    },
    "Catfish": {
        "food_id": 132148,
        "Category": "Meat",
        "vegan": False,
        "vegetarian": False
    },
    "Cauliflower": {
        "food_id": 120954,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Celery": {
        "food_id": 120978,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "CheddarCheese": {
        "food_id": 190576,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Cherry": {
        "food_id": 114638,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Chestnuts": {
        "food_id": 123743,
        "Category": "Nuts",
        "vegan": True,
        "vegetarian": True
    },
    "Cola": {
        "food_id": 139993,
        "Category": "Drink",
        "vegan": True,
        "vegetarian": True
    },
    "CollardGreens": {
        "food_id": 120932,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "CornOil": {
        "food_id": 100003,
        "Category": "Oil",
        "vegan": True,
        "vegetarian": True
    },
    "CornFlour": {
        "food_id": 157503,
        "Category": "Flour",
        "vegan": True,
        "vegetarian": True
    },
    "CottageCheese": {
        "food_id": 190594,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Couscous": {
        "food_id": 157536,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "Crab": {
        "food_id": 132979,
        "Category": "Fish",
        "vegan": False,
        "vegetarian": False
    },
    "Crackers": {
        "food_id": 141145,
        "Category": "Carbohydrate",
        "vegan": True,
        "vegetarian": True
    },
    "Cranberries": {
        "food_id": 114783,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "CreamCheese": {
        "food_id": 190500,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Cucumber": {
        "food_id": 120164,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Dates": {
        "food_id": 114710,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Durian": {
        "food_id": 115715,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Edamame": {
        "food_id": 120185,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "EggNoodles": {
        "food_id": 158789,
        "Category": "Carbohydrate",
        "vegan": True,
        "vegetarian": True
    },
    "Eggplant": {
        "food_id": 120189,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Espresso": {
        "food_id": 139189,
        "Category": "Drink",
        "vegan": True,
        "vegetarian": True
    },
    "GrapeJuice": {
        "food_id": 114954,
        "Category": "Drink",
        "vegan": True,
        "vegetarian": True
    },
    "GrapeFruit": {
        "food_id": 114885,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Grapes": {
        "food_id": 114845,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "GreenOlives": {
        "food_id": 114034,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "GreenBellPeppers": {
        "food_id": 120448,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "GreenPeas": {
        "food_id": 120461,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "GreenTea": {
        "food_id": 139232,
        "Category": "Drink",
        "vegan": True,
        "vegetarian": True
    },
    "GroundGinger": {
        "food_id": 193512,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Guava": {
        "food_id": 114966,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Halibut": {
        "food_id": 132660,
        "Category": "Meat",
        "vegan": False,
        "vegetarian": False
    },
    "HardBoiledEggs": {
        "food_id": 190836,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Hazelnuts": {
        "food_id": 123819,
        "Category": "Nuts",
        "vegan": True,
        "vegetarian": True
    },
    "HoneydewMelon": {
        "food_id": 114001,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "HotChiliPepper": {
        "food_id": 122906,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Hotdog": {
        "food_id": 150803,
        "Category": "Meat",
        "vegan": False,
        "vegetarian": False
    },
    "IcebergLettuce": {
        "food_id": 120205,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "InstantCoffee": {
        "food_id": 139194,
        "Category": "Drink",
        "vegan": True,
        "vegetarian": True
    },
    "Feta": {
        "food_id": 190506,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Figs": {
        "food_id": 114716,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Flaxseed": {
        "food_id": 123119,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "FlaxseedOil": {
        "food_id": 213142,
        "Category": "Oil",
        "vegan": True,
        "vegetarian": True
    },
    "FrenchFriedPotatoes": {
        "food_id": 121758,
        "Category": "Carbohydrate",
        "vegan": True,
        "vegetarian": True
    },
    "Garlic": {
        "food_id": 120194,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Gin": {
        "food_id": 139696,
        "Category": "Alcohol",
        "vegan": True,
        "vegetarian": True
    },
    "GingerRoot": {
        "food_id": 120197,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "GoatCheese": {
        "food_id": 190920,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "GoatMilk": {
        "food_id": 190867,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "GoldenRaisins": {
        "food_id": 114340,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Jackfruit": {
        "food_id": 114981,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Kale": {
        "food_id": 120148,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Kiwi": {
        "food_id": 114993,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Lard": {
        "food_id": 109555,
        "Category": "Fat",
        "vegan": False,
        "vegetarian": False
    },
    "Lemons": {
        "food_id": 114909,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Leeks": {
        "food_id": 120287,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Lentils": {
        "food_id": 135769,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "Lettuce": {
        "food_id": 120208,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Lime": {
        "food_id": 114926,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Lobster": {
        "food_id": 132993,
        "Category": "Seafood",
        "vegan": False,
        "vegetarian": False
    },
    "LongWhiteRice": {
        "food_id": 158884,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "LongBrownRice": {
        "food_id": 157660,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "Loquats": {
        "food_id": 114071,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "MacadamiaNuts": {
        "food_id": 123842,
        "Category": "Nuts",
        "vegan": True,
        "vegetarian": True
    },
    "Mango": {
        "food_id": 114077,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Marshmellow": {
        "food_id": 144897,
        "Category": "Sweets",
        "vegan": False,
        "vegetarian": False
    },
    "MashedPotatoe": {
        "food_id": 122351,
        "Category": "Carbohydrate",
        "vegan": False,
        "vegetarian": True
    },
    "Mayonnaise": {
        "food_id": 109524,
        "Category": "Condiment",
        "vegan": False,
        "vegetarian": True
    },
    "MozzarellaCheese": {
        "food_id": 190536,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "MultigrainBread": {
        "food_id": 172640,
        "Category": "Carbohydrate",
        "vegan": True,
        "vegetarian": True
    },
    "Mussels": {
        "food_id": 132944,
        "Category": "Seafood",
        "vegan": False,
        "vegetarian": False
    },
    "Mustard": {
        "food_id": 193687,
        "Category": "Condiment",
        "vegan": True,
        "vegetarian": True
    },
    "Naan": {
        "food_id": 171470,
        "Category": "Carbohydrate",
        "vegan": True,
        "vegetarian": True
    },
    "Oatmeal": {
        "food_id": 141083,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "Octopus": {
        "food_id": 132149,
        "Category": "Seafood",
        "vegan": False,
        "vegetarian": False
    },
    "Okra": {
        "food_id": 120383,
        "Category": "Seafood",
        "vegan": False,
        "vegetarian": False
    },
    "OliveOil": {
        "food_id": 109608,
        "Category": "Oil",
        "vegan": False,
        "vegetarian": False
    },
    "Onion": {
        "food_id": 120395,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Oranges": {
        "food_id": 114159,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "OrangeJuice": {
        "food_id": 114170,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Oregano": {
        "food_id": 193530,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Oyster": {
        "food_id": 132281,
        "Category": "Seafood",
        "vegan": False,
        "vegetarian": False
    },
    "PalmOil": {
        "food_id": 109614,
        "Category": "Oil",
        "vegan": True,
        "vegetarian": True
    },
    "Pancakes": {
        "food_id": 141329,
        "Category": "Sweets",
        "vegan": False,
        "vegetarian": True
    },
    "Papayas": {
        "food_id": 114127,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "ParmesanCheese": {
        "food_id": 190548,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Parsley": {
        "food_id": 120340,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Parsnips": {
        "food_id": 120343,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Passion": {
        "food_id": 114142,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Pasta": {
        "food_id": 158712,
        "Category": "Carbohydrate",
        "vegan": False,
        "vegetarian": True
    },
    "Peaches": {
        "food_id": 114257,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "PeanutOil": {
        "food_id": 109675,
        "Category": "Oil",
        "vegan": True,
        "vegetarian": True
    },
    "Peanuts": {
        "food_id": 135710,
        "Category": "Nuts",
        "vegan": True,
        "vegetarian": True
    },
    "Pear": {
        "food_id": 114205,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Pecans": {
        "food_id": 123975,
        "Category": "Nuts",
        "vegan": True,
        "vegetarian": True
    },
    "Peppermint": {
        "food_id": 193641,
        "Category": "Spice",
        "vegan": True,
        "vegetarian": True
    },
    "Pineapple": {
        "food_id": 114247,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "PineNuts": {
        "food_id": 123990,
        "Category": "Nuts",
        "vegan": True,
        "vegetarian": True
    }, "Plantains": {
        "food_id": 114380,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Plums": {
        "food_id": 114386,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Pomegrantes": {
        "food_id": 114307,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Pork": {
        "food_id": 127136,
        "Category": "Meat",
        "vegan": False,
        "vegetarian": False
    },
    "Potatoes": {
        "food_id": 121538,
        "Category": "Carbohydrate",
        "vegan": True,
        "vegetarian": True
    },
    "ProvoloneCheese": {
        "food_id": 190654,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Pumpkin": {
        "food_id": 121715,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Quinoa": {
        "food_id": 157960,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "Radish": {
        "food_id": 121736,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Raspberries": {
        "food_id": 114455,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "RedBellPeppers": {
        "food_id": 122912,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "RedWine": {
        "food_id": 139737,
        "Category": "Drink",
        "vegan": True,
        "vegetarian": True
    },
    "RiceNoodles": {
        "food_id": 157951,
        "Category": "Carbohydrate",
        "vegan": True,
        "vegetarian": True
    },
    "Scallop": {
        "food_id": 367279,
        "Category": "Seafood",
        "vegan": False,
        "vegetarian": False
    },
    "Shrimp": {
        "food_id": 132362,
        "Category": "Seafood",
        "vegan": False,
        "vegetarian": False
    },
    "SourCream": {
        "food_id": 190086,
        "Category": "Condiment",
        "vegan": False,
        "vegetarian": True
    },
    "Soybeans": {
        "food_id": 135876,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "Spinach": {
        "food_id": 121820,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Strawberry": {
        "food_id": 114497,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "SweetPotato": {
        "food_id": 121089,
        "Category": "Carbohydrate",
        "vegan": True,
        "vegetarian": True
    },
    "SwissCheese": {
        "food_id": 190679,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Tempeh": {
        "food_id": 135891,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "Tofu": {
        "food_id": 135845,
        "Category": "Grain",
        "vegan": True,
        "vegetarian": True
    },
    "Tomatoes": {
        "food_id": 121036,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Tuna": {
        "food_id": 132812,
        "Category": "Seafood",
        "vegan": False,
        "vegetarian": False
    },
    "TurketMeat": {
        "food_id": 102947,
        "Category": "Meat",
        "vegan": False,
        "vegetarian": False
    },
    "Turnpis": {
        "food_id": 121141,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    },
    "Walnuts": {
        "food_id": 123914,
        "Category": "Nuts",
        "vegan": True,
        "vegetarian": True
    },
    "Watermelon": {
        "food_id": 114427,
        "Category": "Fruit",
        "vegan": True,
        "vegetarian": True
    },
    "Yogurt": {
        "food_id": 190331,
        "Category": "Dairy",
        "vegan": False,
        "vegetarian": True
    },
    "Zucchini": {
        "food_id": 121983,
        "Category": "Vegetable",
        "vegan": True,
        "vegetarian": True
    }   
}
 

with open('food_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()

    for food_name, info in food_options.items():
        food_id = info['food_id']
        scraped_data = scrape_food_page(food_id)
        if scraped_data:
            scraped_data["Name"] = food_name
            # Add Vegan and Vegeterian info
            scraped_data["vegan"] = info['vegan']
            scraped_data["vegetarian"] = info['vegetarian']
            scraped_data["Category"] = info['Category']
            writer.writerow(scraped_data)
            time.sleep(1)

